<a href="https://colab.research.google.com/github/ahmadhajmosa/Machine-Learning-Lab-2020/blob/emir/TimeSeriesPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download the dataset

In [1]:
import pandas as pd

dataset = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv",usecols=[1])

In [4]:
# dataset.Passengers
# dataset.describe()
# dataset.iloc[:2,:]
display(dataset.Passengers.nunique())
display(len(dataset))

118

144

# Plot the time series